In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
from functools import cmp_to_key, partial
import re

import mp_run
import conf_interval

from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error, explained_variance_score

import os

from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from scipy import stats

from multiprocessing import Pool, cpu_count


%load_ext autoreload
%autoreload 2

# regex for number extraction from string
number_pattern =  r'(-?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+-]?\d+)?)'

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

In [2]:
def time_course_compare(item1, item2, valued_time=False):
    rep1, course1 = item1.split('@')
    rep1 = rep1+course1[-1]
    course1 = course1[:-1]
    rep2, course2 = item2.split('@')
    rep2 = rep2+course2[-1]
    course2 = course2[:-1]
    if (valued_time):
        course1 = float(re.search(number_pattern, course1).group(1))
        course2 = float(re.search(number_pattern, course2).group(1))
    if rep1 < rep2:
        return -1
    elif rep1 > rep2:
        return 1
    else:
        if course1 < course2:
            return -1
        elif course1 > course2:
            return 1
        else:
            return 0

In [3]:
exp_df = pd.read_csv('../data/mESC/ExpressionData.csv', index_col=0)


In [4]:
rep_set = set()
time_set = set()

formated_names = []
for name in exp_df.columns:
    name_splits = name.split('_')
    rep_set.add(name_splits[3])
    time_set.add(name_splits[2])
    formated_names.append(name_splits[3]+'@'+name_splits[2])

In [5]:
exp_df.columns = formated_names

In [6]:
sorted_rep_course = sorted(formated_names, key=cmp_to_key(time_course_compare))

In [7]:
exp_df = exp_df[sorted_rep_course]

In [8]:
rep_set = set()
time_set = set()
for name in exp_df.columns:
    name_splits = name.split('@')
    rep_set.add(name_splits[0])
    time_set.add(name_splits[1])

In [9]:
train_source_list = []
train_target_list = []
test_source_list = []
test_target_list = []
for rep in rep_set:
    matching = [s for s in sorted_rep_course if rep in s]
    if len(matching) > 2:
        train_source_list.extend(matching[0:-2])
        train_target_list.extend(matching[1:-1])
        test_source_list.append(matching[-2])
        test_target_list.append(matching[-1])


In [10]:
len(test_target_list)

91

In [11]:
network_df = pd.read_csv('../data/mESC/SubNetwork.csv')
regulator_set = set(network_df['Gene1'])
target_set = set(network_df['Gene2'])

In [12]:
regulator_set = regulator_set.intersection(set(exp_df.index))
target_set = target_set.intersection(set(exp_df.index))
all_gene_set = regulator_set.union(target_set)


In [13]:
len(target_set)

301

In [14]:
network_dict = {target: [] for target in target_set}
for ind, row in network_df.iterrows():
    if (row['Gene1'] in regulator_set) and (row['Gene2'] in target_set):
        network_dict[row['Gene2']].append(row['Gene1'])

In [15]:
key_list = []
value_list = []
regulator_set = set()
target_set = set()
for key in network_dict.keys():
    if (len(network_dict[key]) > 0):
        key_list.append(key)
        target_set.add(key)
        value_list.append("; ".join(network_dict[key]))
        for regulator in network_dict[key]:
            regulator_set.add(regulator)
all_gene_set = regulator_set.union(target_set)

In [16]:
network_df = pd.DataFrame(index=key_list)
network_df['tf_list'] = value_list

In [17]:
exp_df = exp_df.apply(stats.zscore, axis=0)

In [18]:
train_source = exp_df[train_source_list]
train_target = exp_df[train_target_list]
test_source = exp_df[test_source_list]
test_target = exp_df[test_target_list]

target_df = pd.concat([train_target, test_target], axis=1)
source_df = pd.concat([train_source, test_source], axis=1)






In [22]:
for a in list(target_set):
    if len(network_df.loc[a]['tf_list'].split('; ')) < 1 or network_df.loc[a]['tf_list'] == a:
        target_set.remove(a)
    

In [23]:
target_gene_list = list(target_set)
target_exp = target_df
X = source_df.loc[list(regulator_set)]
tf_list = list(regulator_set)

In [24]:
mp_calc = mp_run.MpCalc(target_gene_list, target_exp, X, network_df, train_source.loc[tf_list], train_target, test_source.loc[tf_list], test_target)

In [25]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    r = list(tqdm(p.imap(mp_calc.full_comp, range(iter_length)), total=iter_length))

100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [00:14<00:00, 20.47it/s]


In [26]:
r = np.array(r)
out_df = pd.DataFrame(index=target_gene_list)
out_df['rf_score'] = r[:, 0]
out_df['linear_score'] = r[:, 1]
out_df['gs_rf_score'] = r[:, 2]
out_df['gs_linear_score'] = r[:, 3]
out_df['rf_with_linear_top_features_score'] = r[:, 4]
out_df['linear_with_rf_top_features_score'] = r[:, 5]
out_df['rf_rmse'] = r[:, 6]
out_df['linear_rmse'] = r[:, 7]
out_df['gs_rf_rmse'] = r[:, 8]
out_df['gs_linear_rmse'] = r[:, 9]
out_df['rf_with_linear_top_features_rmse'] = r[:, 10]
out_df['linear_with_rf_top_features_rmse'] = r[:, 11]


In [80]:
rf_conf_interval = conf_interval.conf_interval_calc(list(out_df['rf_score'].values))
print('('+', '.join([ '%.3f' % elem for elem in rf_conf_interval[2:] ])+')')
linear_conf_interval = conf_interval.conf_interval_calc(list(out_df['linear_score'].values))
print('('+', '.join([ '%.3f' % elem for elem in linear_conf_interval[2:] ])+')')
gs_rf_conf_interval = conf_interval.conf_interval_calc(list(out_df['gs_rf_score'].values))
print('('+', '.join([ '%.3f' % elem for elem in gs_rf_conf_interval[2:] ])+')')
gs_linear_conf_interval = conf_interval.conf_interval_calc(list(out_df['gs_linear_score'].values))
print('('+', '.join([ '%.3f' % elem for elem in gs_linear_conf_interval[2:] ])+')')
rf_with_linear_top_features_conf_interval = conf_interval.conf_interval_calc(list(out_df['rf_with_linear_top_features_score'].values))
print('('+', '.join([ '%.3f' % elem for elem in rf_with_linear_top_features_conf_interval[2:] ])+')')
linear_with_rf_top_features_conf_interval = conf_interval.conf_interval_calc(list(out_df['linear_with_rf_top_features_score'].values))
print('('+', '.join([ '%.3f' % elem for elem in linear_with_rf_top_features_conf_interval[2:] ])+')')


(-0.922, -0.735)
(-1.960, -1.383)
(-1.692, -1.282)
(-2.209, -1.519)
(-1.171, -0.950)
(-1.130, -0.862)


In [81]:
rf_conf_interval_rmse = conf_interval.conf_interval_calc(list(out_df['rf_rmse'].values))
print('('+', '.join([ '%.3f' % elem for elem in rf_conf_interval_rmse[2:] ])+')')
linear_conf_interval_rmse = conf_interval.conf_interval_calc(list(out_df['linear_rmse'].values))
print('('+', '.join([ '%.3f' % elem for elem in linear_conf_interval_rmse[2:] ])+')')
gs_rf_conf_interval_rmse = conf_interval.conf_interval_calc(list(out_df['gs_rf_rmse'].values))
print('('+', '.join([ '%.3f' % elem for elem in gs_rf_conf_interval_rmse[2:] ])+')')
gs_linear_conf_interval_rmse = conf_interval.conf_interval_calc(list(out_df['gs_linear_rmse'].values))
print('('+', '.join([ '%.3f' % elem for elem in gs_linear_conf_interval_rmse[2:] ])+')')
rf_with_linear_top_features_conf_interval_rmse = conf_interval.conf_interval_calc(list(out_df['rf_with_linear_top_features_rmse'].values))
print('('+', '.join([ '%.3f' % elem for elem in rf_with_linear_top_features_conf_interval_rmse[2:] ])+')')
linear_with_rf_top_features_conf_interval_rmse = conf_interval.conf_interval_calc(list(out_df['linear_with_rf_top_features_rmse'].values))
print('('+', '.join([ '%.3f' % elem for elem in linear_with_rf_top_features_conf_interval_rmse[2:] ])+')')


(0.442, 0.492)
(0.486, 0.535)
(0.493, 0.548)
(0.496, 0.549)
(0.467, 0.518)
(0.450, 0.500)


In [82]:
out_df.mean()[:6]

rf_score                            -0.822921
linear_score                        -1.630093
gs_rf_score                         -1.469246
gs_linear_score                     -1.815687
rf_with_linear_top_features_score   -1.056953
linear_with_rf_top_features_score   -0.986664
dtype: float64

In [83]:
out_df.mean()[6:]

rf_rmse                             0.466460
linear_rmse                         0.510472
gs_rf_rmse                          0.520083
gs_linear_rmse                      0.522475
rf_with_linear_top_features_rmse    0.491276
linear_with_rf_top_features_rmse    0.473984
dtype: float64

In [84]:
from itertools import combinations
model_combs = list(combinations(out_df.columns[:6], 2))

In [85]:
for a, b in model_combs:
    t, p = stats.ttest_rel(out_df[a], out_df[b])
    c, d, lower, upper = conf_interval.conf_interval_calc(list(out_df[a]-out_df[b]))
    if (p > 0.05):
        print('{} and {} don\'t have statistically different performance'.format(a, b))
        continue
    if (t > 0):
        print('{} has statisically better performance than {}, with p-val of {}'.format(a, b, p))
        print('confidence interval: ({:.3f}, {:.3f})'.format(lower, upper))
    else:
        print('{} has statisically better performance than {}, with p-val of {}'.format(b, a, p))
        print('confidence interval: ({:.3f}, {:.3f})'.format(lower, upper))

rf_score has statisically better performance than linear_score, with p-val of 2.0174658909807522e-07
confidence interval: (0.593, 1.099)
rf_score has statisically better performance than gs_rf_score, with p-val of 9.153268740522008e-11
confidence interval: (0.508, 0.822)
rf_score has statisically better performance than gs_linear_score, with p-val of 1.457397541011229e-07
confidence interval: (0.736, 1.337)
rf_score has statisically better performance than rf_with_linear_top_features_score, with p-val of 1.265123865281187e-13
confidence interval: (0.187, 0.286)
rf_score has statisically better performance than linear_with_rf_top_features_score, with p-val of 0.0025694463934270917
confidence interval: (0.088, 0.264)
linear_score and gs_rf_score don't have statistically different performance
linear_score and gs_linear_score don't have statistically different performance
rf_with_linear_top_features_score has statisically better performance than linear_score, with p-val of 0.00023581153675

In [27]:
iter_length = len(target_gene_list)
with Pool(cpu_count()) as p:
    feature_num_r = list(tqdm(p.imap(mp_calc.top_feature_num, range(iter_length)), total=iter_length))

100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [00:09<00:00, 32.31it/s]


In [31]:
feature_num_r = np.array(feature_num_r)
out_df['rf_top_feature_num'] = feature_num_r[:, 0]
out_df['linear_top_feature_num'] = feature_num_r[:, 1]
out_df.to_csv('mesc_network_v_model.csv')

In [30]:
np.std(feature_num_r[:, 0])

0.8924062353485036

In [32]:
print(len(target_gene_list), len(regulator_set))

300 46


In [37]:
exp_df.shape

(547, 421)